## Descargar datos - 1pts

Buscar el la pargina https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived.html, los datos mensuales de precipitations y temperatura

Descaregar estos fichieros, y guardar los en una carpeta adecuada.



## Temperatura - precip - 6pts - tiempo maximo 2h

1) extraer los datos de precipitation y temperatura sobre europa (como teneis que hacer lo dos veces vuestro codigo quedara mejor si escribeis una funccion) 

In [3]:
#import netcdf library
from netCDF4 import Dataset as nc 
#import date conversor netcdf
from netCDF4 import num2date, date2num 
from datetime import date, timedelta, datetime
import numpy as np
import os

In [8]:
### This cell is all commented as it is only useful if the user wants to check fetures of the file. ###

#file_Ts = nc("air.2m.mon.mean.nc", "r") # open the netcdf file
#file_Precip = nc("prate.sfc.mon.mean.nc", "r") 
#print(file_Ts.variables.get("air"))
#print(file_Ts.variables.get("air").shape)
#print(file_Precip.variables.get("prate"))
#print(file_Precip.variables.get("prate").shape)
#print(file_Ts.variables.keys())
#print(file_Precip.dimensions.keys())
#print(file_Precip.variables.keys())


ei <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    description: Data is from NMC initialized reanalysis
(4x/day).  It consists of T42  variables interpolated to
pressure surfaces from model (sigma) surfaces.
    platform: Model
    Conventions: COARDS
    NCO: 20121013
    history: Mon Jul  5 23:55:54 1999: ncrcat prate.mon.mean.nc /Datasets/ncep.reanalysis.derived/surface_gauss/prate.mon.mean.nc /dm/dmwork/nmc.rean.ingest/combinedMMs/prate.mon.mean.nc
/home/hoop/crdc/cpreanjuke2farm/cpreanjuke2farm Mon Oct 23 21:04:20 1995 from prate.sfc.gauss.85.nc
created 95/03/13 by Hoop (netCDF2.3)
Converted to chunked, deflated non-packed NetCDF4 2014/09
    title: monthly mean prate.sfc from the NCEP Reanalysis
    References: http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived.html
    dataset_title: NCEP-NCAR Reanalysis 1
    dimensions(sizes): lat(94), lon(192), time(861)
    variables(dimensions): float32 lat(lat), float32 

In [4]:
def read_var_from_file (file_name, variable, lon_lat, lon_lat_name):
    """
    function that reads a file, and limits a variable to a 
    region of the earth globe (latitude and longitude)
    file_name: name of the file (as saved in the folder)
    variable: name of the variable that has to be boxed
    lon_lat: Boundaries of latitude and longitude for the region studied (box)
    lon_lat_name: Array of the names that the file uses for "latitude" and "longitude" variables
    """
    file = nc(file_name, "r")
    
    # extract the longitude and latitude variable values from the file
    lon = file.variables[lon_lat_name[0]][:]
    lat = file.variables[lon_lat_name[1]][:]
    
    # Make all the longitudes fall in the range of 0-360 degrees 
    # As the files don't have longitude organized from -180 to 180 deg
    lon_lat = np.array(lon_lat)
    lon_lat[:2] = (lon_lat[:2] + 360)%360
    
    # Bound the latitude between the latitude boundaries
    lat_box = np.where((lat > lon_lat[2])&(lat < lon_lat[3]))[0]
    
    if lon_lat[0]>lon_lat[1]:
        # calculate values at the west of Greenwitch
        lon_box1 = np.where((lon >= lon_lat[0]))[0]
        # calculate values at the east of Greenwitch
        lon_box2 = np.where((lon <= lon_lat[1]))[0]
        # extract data for the west of Greenwitch
        geo_box1 = file.variables[variable][:,lat_box,lon_box1]
        # extract data for the east of Greenwitch
        geo_box2 = file.variables[variable][:,lat_box,lon_box2]
        # concatenate both extracted data into a single box
        geo_box = np.concatenate((geo_box1, geo_box2), axis=2)
    else:
        # Bound the longitude between the longitude boundaries
        lon_box = np.where((lon >= lon_lat[0])&(lon <= lon_lat[1]))[0]
        # create a data varaible inside the box defined
        geo_box = file.variables[variable][:,lat_box,lon_box]
    return (geo_box)

In [5]:
# Box of Latitude and Longitude for Europe (lon1, lon2, lat1, lat2)
lon_lat = [-20, 60, 30, 70]
# The variables latitude and longitude are referenced with the following names 
lon_lat_name = ["lon","lat"]
# Call read_var_from_file function for both T2m and P-rate files 
Precip_NCEP = read_var_from_file("prate.sfc.mon.mean.nc","prate",lon_lat,lon_lat_name)
T2m_NCEP = read_var_from_file("air.2m.mon.mean.nc","air",lon_lat,lon_lat_name)
# Show shape of both files
Precip_NCEP.shape, T2m_NCEP.shape


(861, 94, 192)
3
(861, 94, 192)
3


((861, 21, 43), (861, 21, 43))

In [ ]:
file_Ts.close()
file_Precip.close()

### Temperatura ERA4 - 3pt

In [22]:
### This cell is all commented as it is only useful if the user wants to check fetures of the file. ###

#fileERA = nc("ERA4.nc", "r")
#print("ei",fileERA,"END")
#print(fileERA.variables.get("t2m"))

ei <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2019-10-28 11:56:31 GMT by grib_to_netcdf-2.14.0: /opt/ecmwf/eccodes/bin/grib_to_netcdf -o /cache/data1/adaptor.mars.internal-1572263600.6175466-6392-6-d0f3e618-d8ee-4016-bd57-14c2ff68a5bd.nc /cache/tmp/d0f3e618-d8ee-4016-bd57-14c2ff68a5bd-adaptor.mars.internal-1572263600.6183522-6392-3-tmp.grib
    dimensions(sizes): longitude(1440), latitude(721), time(488)
    variables(dimensions): >f4 longitude(longitude), >f4 latitude(latitude), >i4 time(time), >i2 t2m(time,latitude,longitude)
    groups: 
 END
<class 'netCDF4._netCDF4.Variable'>
>i2 t2m(time, latitude, longitude)
    scale_factor: 0.0017631979796170203
    add_offset: 257.70367009778755
    _FillValue: -32767
    missing_value: -32767
    units: K
    long_name: 2 metre temperature
unlimited dimensions: 
current shape = (488, 721, 1440)
filling on


2) En esta pagina esta disponible la reanalysis ERA4 https://climate.copernicus.eu/climate-reanalysis, descarregar las temperaturas a dos metros mensual (2m temperature) y llegir estos datos sobre Europa

In [21]:
# Box of Latitude and Longitude for Europe (lon1, lon2, lat1, lat2)
lon_lat = [-20, 60, 30, 70]
# The variables latitude and longitude are referenced with the following names 
lon_lat_name = ["longitude","latitude"]
# Call read_var_from_file function for T2m file (ERA4)
T2m_ERA4 = read_var_from_file("ERA4.nc","t2m",lon_lat,lon_lat_name)
T2m_ERA4.shape     

(488, 159, 321)

### Bonus - comparar la reanalysis NCEP y ERAinterim

Comparar de la manera que os paresca mas adequada los datos de NCEP y ERA interim.